# Pizza Object Generator

This notebook creates ten imaginative pizza concepts using Groq's text generation API and renders
matching imagery through Together AI's FLUX model. Each pizza is saved as a JSON descriptor along
with its corresponding PNG image. Run the cells top-to-bottom inside Google Colab.

## API key setup

Set the `GROQ_API_KEY` and `TOGETHER_API_KEY` environment variables before executing the generation cell.
You can either store them in Colab secrets, or run the helper cell below to enter them securely
with `getpass`.

In [ ]:
# Run this once per runtime to install dependencies.
!pip install --quiet groq together

In [ ]:
import base64import jsonimport osimport textwrapimport timefrom dataclasses import dataclass, asdictfrom pathlib import Pathfrom typing import Iterable, List, Optionalfrom groq import Groqfrom together import TogetherDEFAULT_TEXT_MODEL = "openai/gpt-oss-120b"DEFAULT_IMAGE_MODEL = "black-forest-labs/FLUX.1-schnell-Free"IMAGE_REQUESTS_PER_MINUTE = 6

In [ ]:
@dataclassclass PizzaConcept:    title: str    description: str    image_prompt: str    image_path: Optional[Path] = None    def to_serializable(self) -> dict:        data = asdict(self)        if self.image_path is not None:            data["image_path"] = str(self.image_path)        return datadef build_groq_client(api_key: Optional[str] = None) -> Groq:    key = api_key or os.environ.get("GROQ_API_KEY")    if not key:        raise RuntimeError(            "GROQ_API_KEY is required. Set it via environment variable or update the key entry cell."        )    return Groq(api_key=key)def build_together_client(api_key: Optional[str] = None) -> Together:    key = api_key or os.environ.get("TOGETHER_API_KEY")    if not key:        raise RuntimeError(            "TOGETHER_API_KEY is required. Set it via environment variable or update the key entry cell."        )    return Together(api_key=key)def pizza_prompt(num_pizzas: int) -> str:    return textwrap.dedent(        f"""        You are a culinary creative assistant. Invent {num_pizzas} distinct pizzas.        Return **only** valid JSON following this exact schema:        {{          "pizzas": [            {{              "title": "string",              "description": "1-2 sentences describing the pizza",              "image_prompt": "Imagery instructions for an image generator"            }}          ]        }}        Requirements:        - Use unique ingredients, styles, and inspirations for each pizza.        - Make sure the `image_prompt` is concise but vivid enough for FLUX image generation.        - Stay grounded in appetizing, realistic concepts (avoid fantastical or unsafe ingredients).        - Do not include markdown fences or commentary outside the JSON object.        """    ).strip()def fetch_pizza_concepts(client: Groq, count: int) -> List[PizzaConcept]:    response = client.chat.completions.create(        model=DEFAULT_TEXT_MODEL,        temperature=0.8,        max_tokens=800,        response_format={"type": "json_object"},        messages=[            {"role": "system", "content": "You produce structured culinary ideation."},            {"role": "user", "content": pizza_prompt(count)},        ],    )    content = response.choices[0].message.content    payload = json.loads(content)    pizzas: List[PizzaConcept] = []    for item in payload.get("pizzas", []):        pizzas.append(            PizzaConcept(                title=item["title"].strip(),                description=item["description"].strip(),                image_prompt=item.get("image_prompt", item["description"]).strip(),            )        )    if len(pizzas) != count:        raise ValueError(f"Expected {count} pizzas but received {len(pizzas)}. Raw payload: {payload}")    return pizzasdef ensure_output_dirs(base_dir: Path) -> tuple[Path, Path]:    json_dir = base_dir / "pizza_json"    image_dir = base_dir / "pizza_images"    json_dir.mkdir(parents=True, exist_ok=True)    image_dir.mkdir(parents=True, exist_ok=True)    return json_dir, image_dirdef slugify(value: str) -> str:    return "-".join(        "".join(ch for ch in value.lower() if ch.isalnum() or ch in {" ", "-"}).split()    )def generate_images(    client: Together,    pizzas: Iterable[PizzaConcept],    image_dir: Path,    requests_per_minute: int = IMAGE_REQUESTS_PER_MINUTE,) -> None:    interval = 60.0 / max(1, requests_per_minute)    last_request_ts: Optional[float] = None    for concept in pizzas:        now = time.monotonic()        if last_request_ts is not None:            elapsed = now - last_request_ts            if elapsed < interval:                time.sleep(interval - elapsed)        last_request_ts = time.monotonic()        prompt = f"Food photography of {concept.title} pizza. {concept.image_prompt}"        response = client.images.generate(            prompt=prompt,            model=DEFAULT_IMAGE_MODEL,            steps=10,            n=1,        )        image_b64 = response.data[0].b64_json        image_bytes = base64.b64decode(image_b64)        slug = slugify(concept.title)        image_path = image_dir / f"{slug}.png"        with image_path.open("wb") as fp:            fp.write(image_bytes)        concept.image_path = image_pathdef persist_pizzas(pizzas: Iterable[PizzaConcept], json_dir: Path) -> None:    for concept in pizzas:        slug = slugify(concept.title)        json_path = json_dir / f"{slug}.json"        with json_path.open("w", encoding="utf-8") as fp:            json.dump(concept.to_serializable(), fp, indent=2, ensure_ascii=False)def generate_pizza_batch(    output_root: Path,    count: int = 10,    groq_key: Optional[str] = None,    together_key: Optional[str] = None,) -> List[PizzaConcept]:    groq_client = build_groq_client(groq_key)    together_client = build_together_client(together_key)    pizzas = fetch_pizza_concepts(groq_client, count)    json_dir, image_dir = ensure_output_dirs(output_root)    generate_images(together_client, pizzas, image_dir)    persist_pizzas(pizzas, json_dir)    return pizzasdef display_pizzas(pizzas: Iterable[PizzaConcept]) -> None:    from IPython.display import Image, display    for concept in pizzas:        display({            "title": concept.title,            "description": concept.description,            "image": Image(filename=str(concept.image_path)) if concept.image_path else None,        })

In [ ]:
# Enter your API keys securely for the current session.# These values are not stored once the runtime shuts down.import getpassimport osif not os.environ.get("GROQ_API_KEY"):    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter GROQ API key: ")if not os.environ.get("TOGETHER_API_KEY"):    os.environ["TOGETHER_API_KEY"] = getpass.getpass("Enter Together API key: ")

In [ ]:
# Generate pizzas, persist JSON + images, and display the results.output_root = Path.cwd() / "pizza_outputs"output_root.mkdir(parents=True, exist_ok=True)pizzas = generate_pizza_batch(output_root, count=10)print(f"Saved pizza metadata to {output_root / 'pizza_json'}")print(f"Saved pizza images to {output_root / 'pizza_images'}")try:    display_pizzas(pizzas)except Exception as exc:    print("Display skipped:", exc)